In [1]:
%matplotlib notebook
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.io.fits as pf
from astropy.stats import mad_std
from astropy.visualization import ZScaleInterval
import ccdproc as ccdp
from astropy import units as u

from astropy.nddata import CCDData
from iraf_um import imstat, inv_median

import warnings
from astropy.utils.exceptions import AstropyWarning

***Path***

In [2]:
path = 'E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/'

path_dict = { 'bias' : '2024-09-23-lyon-mtp/bias/', 'flat_r' : '2024-09-24-lyon-mtp/flat/filtre_r/', 'flat_g' : '2024-09-24-lyon-mtp/flat/filtre_g/',
              'dark' : '2024-09-23-lyon-mtp/dark/', 'baseline_23' : '2024-09-23-lyon-mtp/baseline/', 'baseline_24' : '2024-09-24-lyon-mtp/baseline/',
              'obs_sec_r' : '2024-09-23-lyon-mtp/observation_secondaire/', 'obs_sec_g' : '2024-09-24-lyon-mtp/observation_secondaire/',
              'obs_pri_r' : '2024-09-24-lyon-mtp/observation_primaire/', 'baseline_23_t' : '2024-09-23-lyon-mtp_t/baseline/' , 
              'baseline_24_t' : '2024-09-24-lyon-mtp_t/baseline/', 'obs_sec_r_t' : '2024-09-23-lyon-mtp_t/observation_secondaire/',
              'obs_sec_g_t' : '2024-09-24-lyon-mtp_t/observation_secondaire/', 'obs_pri_r_t' : '2024-09-24-lyon-mtp_t/observation_primaire/' }

print(path+path_dict['bias'])

E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp/bias/


***CCD Char***

In [3]:
ccd_gain = 1.47 * u.electron / u.adu
ccd_ron = 11 * u.electron

***Master bias***

In [5]:
ifc_raw = ccdp.ImageFileCollection(path+path_dict['bias'])
ifc_bias = ifc_raw.filter(imagetyp='Bias Frame')
warnings.filterwarnings("ignore", category=AstropyWarning)
master_bias = ccdp.combine(ifc_bias.files_filtered(include_path=True), unit='adu', combine='average', mem_limit=9000.e6)
warnings.filterwarnings("default")
master_bias.meta['COMBINED'] = True
master_bias.data = np.clip(master_bias.data, 0, np.iinfo(np.int16).max).astype('int16')
master_bias.write(path+'master_bias.fit', hdu_mask=None, hdu_uncertainty=None, overwrite=True)

***Master flat***

In [6]:
ifc_raw = [ ccdp.ImageFileCollection(path+path_dict['flat_r']), ccdp.ImageFileCollection(path+path_dict['flat_g']) ]
ifc_flatH = [ ifc_raw[0].filter(imagetyp='Flat Field'), ifc_raw[1].filter(imagetyp='Flat Field') ]
master_flatH = [None,None]
for i in range(len(ifc_flatH)):
    warnings.filterwarnings("ignore", category=AstropyWarning)
    master_flatH[i] = ccdp.combine(ifc_flatH[i].files_filtered(include_path=True), unit='adu', combine='average', mem_limit=9000.e6)
    warnings.filterwarnings("default")
    master_flatH[i].meta['COMBINED'] = True
    master_flatH[i] = ccdp.subtract_bias(master_flatH[i], master_bias)
    master_flatH[i].data = np.clip(master_flatH[i].data, 0, np.iinfo(np.int16).max).astype('int16')

master_flatH[0].write(path + 'master_flatH_r.fit', hdu_mask=None, hdu_uncertainty=None, overwrite=True)
master_flatH[1].write(path + 'master_flatH_g.fit', hdu_mask=None, hdu_uncertainty=None, overwrite=True)

***Master dark***

In [9]:
ifc_raw = ccdp.ImageFileCollection(path+path_dict['dark'])
ifc_dark = ifc_raw.filter()
warnings.filterwarnings("ignore", category=AstropyWarning)
master_dark = ccdp.combine(ifc_dark.files_filtered(include_path=True), unit='adu', combine='average', mem_limit=9000.e6)
warnings.filterwarnings("default")
master_dark.meta['COMBINED'] = True
master_dark.data = np.clip(master_dark.data, 0, np.iinfo(np.int16).max).astype('int16')
master_dark.write(path+'master_dark.fit', hdu_mask=None, hdu_uncertainty=None, overwrite=True)

In [4]:
warnings.filterwarnings("ignore", category=AstropyWarning)
master_bias = ccdp.CCDData.read(path + 'master_bias.fit', unit='adu')
master_bias.to_hdu().info()
print('\n')
master_flatH_g = ccdp.CCDData.read(path + 'master_flatH_g.fit', unit='adu')
master_flatH_g.to_hdu().info()
print('\n')
master_flatH_r = ccdp.CCDData.read(path + 'master_flatH_r.fit', unit='adu')
master_flatH_r.to_hdu().info()
print('\n')
master_dark = ccdp.CCDData.read(path+'master_dark.fit', unit='adu')
master_flatH_r.to_hdu().info()
warnings.filterwarnings("default")

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      37   (3072, 2047)   int16   


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      40   (3072, 2047)   int16   


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      40   (3072, 2047)   int16   


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.

***Traitement image***

In [6]:
input = ['baseline_23', 'baseline_24', 'obs_sec_r', 'obs_sec_g','obs_pri_r' ]
output = ['baseline_23_t', 'baseline_24_t', 'obs_sec_r_t', 'obs_sec_g_t','obs_pri_r_t' ]

collection = input[0]

ifc_raw = ccdp.ImageFileCollection(path+path_dict[collection])
ifc_sci = ifc_raw.filter(imagetyp='Light Frame')

image = ifc_sci.summary['file'][0]

warnings.filterwarnings("ignore", category=AstropyWarning)
name = os.path.split(path+path_dict[collection]+image)[1]
print(path+path_dict[output[input.index(collection)]])
ccd_sci = ccdp.CCDData.read(image, unit='adu')
ccd_sci_b = ccdp.subtract_bias(ccd_sci, master_bias)
ccd_sci_bf = ccdp.flat_correct(ccd_sci_b, master_flatH_r)
ccd_sci_bfd = ccdp.subtract_dark(ccd_sci_bf, master_dark, exposure_time=10, exposure_unit=u.second)
ccd_sci_bfdg = ccdp.gain_correct(ccd_sci_bfd, ccd_gain)
ccd_sci_bfdg.data = np.clip(ccd_sci_bfdg.data, 0, np.iinfo(np.int16).max).astype('int16')
ccd_sci_bfdg.write(path+path_dict[output[input.index(collection)]]+name, hdu_mask=None, hdu_uncertainty=None, overwrite=True)
warnings.filterwarnings("default")

E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/


In [ ]:
input = ['baseline_23', 'baseline_24', 'obs_sec_r', 'obs_sec_g','obs_pri_r' ]
output = ['baseline_23_t', 'baseline_24_t', 'obs_sec_r_t', 'obs_sec_g_t','obs_pri_r_t' ]
for collection in input:
    ifc_raw = ccdp.ImageFileCollection(path+path_dict[collection])
    ifc_sci = ifc_raw.filter(imagetyp='Light Frame')
    for image in ifc_sci.summary['file']:
        warnings.filterwarnings("ignore", category=AstropyWarning)
        name = os.path.split(path+path_dict[collection]+image)[1]
        print(path+path_dict[output[input.index(collection)]])
        ccd_sci = ccdp.CCDData.read(image, unit='adu')
        ccd_sci_b = ccdp.subtract_bias(ccd_sci, master_bias)
        ccd_sci_bf = ccdp.flat_correct(ccd_sci_b, master_flatH_r)
        ccd_sci_bfd = ccdp.subtract_dark(ccd_sci_bf, master_dark, exposure_time=10, exposure_unit=u.second)
        ccd_sci_bfdg = ccdp.gain_correct(ccd_sci_bfd, ccd_gain)
        ccd_sci_bfdg.data = np.clip(ccd_sci_bfdg.data, 0, np.iinfo(np.int16).max).astype('int16')
        ccd_sci_bfdg.write(path+path_dict[output[input.index(collection)]]+name, hdu_mask=None, hdu_uncertainty=None, overwrite=True)
        warnings.filterwarnings("default")

E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/
E:/Documents/Cours_et_administratif/FAC/Master/M2/ohp/data/2024-09-23-lyon-mtp_t/baseline/